In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import tensorflow as tf
from transformers import DistilBertTokenizer, TFDistilBertModel
distil_bert = 'distilbert-base-uncased'

In [ ]:
df_original = pd.read_excel("primary_intent_up_english_after_Datathon2.xlsx")

In [ ]:
df = df_original[:1000]

In [ ]:
df.head()

,Text,Intent
0,Sec 18,Store
1,Store,Store
2,Store Near Me,Store
3,Stores Near Me,Store
4,Nearest Store to me,Store


In [ ]:
df.sort_values('Text',inplace=True, ascending=True)
duplicated_text_series = df.duplicated('Text', keep = 'first')
df = df[~duplicated_text_series]
print("Total number of text after removing duplicates:", df.shape[0])

Total number of text after removing duplicates: 841


/home/deepakchawala/PycharmProjects/HBChawlaProjects/geekeeprojectModules/texttocode/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df.head(20)

,Text,Intent
442,Appointment,Store
29,Assisted shopping is there ?,Store
918,Band-collar shirt for women,Product
807,Band-collar tshirt for men,Product
134,Black,Product
132,Black Shoe,Product
193,Black loafers,Product
167,Black shoes,Product
219,Black slippers,Product
551,Blue color flat sandals,Product


In [ ]:
import string

In [ ]:
text = "stores near me?"

In [ ]:
def punctuation_removal(text):
  text = [i for i in text if i not in string.punctuation]
  text = "".join(text)
  return text

In [ ]:
df["Text"] = df["Text"].apply(punctuation_removal)

In [ ]:
df["Intent"].value_counts()

Product    667
Store      106
Loyalty     68
Name: Intent, dtype: int64

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained(distil_bert)
model = TFDistilBertModel.from_pretrained(distil_bert)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_transform', 'activation_13', 'vocab_projector', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [ ]:
tokenized = df["Text"].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [ ]:
max_len = 50
padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [ ]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(841, 50)

In [ ]:
input_ids =tf.convert_to_tensor(padded)
attention_mask = tf.convert_to_tensor(attention_mask)

In [ ]:
last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()

In [ ]:
labels = df["Intent"]

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, random_state=42)

In [ ]:
#from sklearn.naive_bayes import MultinomialNB
#clf = MultinomialNB()
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn import svm
# lr_clf = GaussianNB()
# lr_clf.fit(train_features, train_labels)
# lr_clf = MultinomialNB().fit(train_features,train_labels)
lr_clf=LinearSVC(class_weight='balanced',random_state=42)
#lr_clf=svm.SVC(kernel = 'poly', C = 10, probability=True,class_weight='balanced')
lr_clf.fit(train_features, train_labels)

/home/deepakchawala/PycharmProjects/HBChawlaProjects/geekeeprojectModules/texttocode/venv/lib/python3.6/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(class_weight='balanced', random_state=42)

In [ ]:
#lr_clf2=LinearSVC(class_weight='balanced')
#lr_clf2.fit(train_features, train_labels)

In [ ]:
#lr_clf = LogisticRegression(multi_class = "multinomial", random_state=42)
#lr_clf.fit(train_features, train_labels)

In [ ]:
lr_clf.score(test_features, test_labels)

1.0

In [ ]:
y_pred=lr_clf.predict(test_features)

In [ ]:
#y_pred=lr_clf2.predict(test_features)

In [ ]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(test_labels, y_pred)
print(confusion_matrix)
from sklearn.metrics import classification_report
print(classification_report(test_labels, y_pred))

print("Accuracy:",metrics.accuracy_score(test_labels, y_pred))

[[ 14   0   0]
 [  0 174   0]
 [  0   0  23]]
              precision    recall  f1-score   support

     Loyalty       1.00      1.00      1.00        14
     Product       1.00      1.00      1.00       174
       Store       1.00      1.00      1.00        23

    accuracy                           1.00       211
   macro avg       1.00      1.00      1.00       211
weighted avg       1.00      1.00      1.00       211

Accuracy: 1.0


In [ ]:
#confusion_matrix = confusion_matrix(test_labels, y_pred)
#print(confusion_matrix)
#print(classification_report(test_labels, y_pred))
#print("Accuracy:",metrics.accuracy_score(test_labels, y_pred))

In [ ]:
def feature_gen(text):
    text = [i for i in text if i not in string.punctuation]
    text = "".join(text)
    dummy = tokenizer.encode(text,add_special_tokens=True)
    dummy_padded = np.array([dummy + [0]*(max_len-len(dummy))])
    dummy_attention_mask = np.where(dummy_padded != 0, 1, 0)
    dummy_input_ids =tf.convert_to_tensor(dummy_padded)
    dummy_attention_mask = tf.convert_to_tensor(dummy_attention_mask)
    dummy_last_hidden_states = model(dummy_input_ids, attention_mask=dummy_attention_mask)
    dummy_features = dummy_last_hidden_states[0][:,0,:].numpy()
    return dummy_features

In [ ]:
#from sklearn.linear_model import SGDClassifier
#from sklearn.calibration import CalibratedClassifierCV

In [ ]:
#clf = SGDClassifier(class_weight='balanced', penalty='l2', loss='hinge', random_state=42)
#clf.fit(train_features, train_labels)
#calib_clf = CalibratedClassifierCV(clf, method="sigmoid")
#calib_clf.fit(train_features, train_labels)

In [ ]:
#calib_clf.score(test_features, test_labels)

In [ ]:
train_labels.value_counts()

Product    493
Store       83
Loyalty     54
Name: Intent, dtype: int64

In [ ]:
#calib_clf.classes_

Out[80]: array(['Confused', 'FAQs', 'Greetings', 'Invoice', 'Loyalty', 'Product',
 'Shoe Size', 'Store'], dtype='<U9')

In [ ]:
#calib_clf.predict_proba(feature_gen("bataclub")).argmax()

In [ ]:
#calib_clf.predict(feature_gen("bataclub"))

In [ ]:
# import pickle
# filename = 'finalized_model_english_after_Datathon.sav'
# lr_clf = pickle.load(open(filename, 'rb'))

In [ ]:
lr_clf.classes_

array(['Loyalty', 'Product', 'Store'], dtype=object)

In [ ]:
lr_clf.predict(feature_gen("bata shoe size"))

array(['Product'], dtype=object)

In [ ]:
#lr_clf.predict_proba(feature_gen("namas")).max()

--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-3952795988958848> in <module> 
 ----> 1 lr_clf . predict_proba ( feature_gen ( "namas" ) ) . max ( ) 

 AttributeError : 'LinearSVC' object has no attribute 'predict_proba'

In [ ]:
lr_clf.predict(feature_gen("nadia"))[0]

'Store'

In [ ]:
lr_clf.predict(feature_gen("store near me"))[0]

'Store'

In [ ]:
df[df['Intent']=='Store']

,Text,Intent
442,Appointment,Store
29,Assisted shopping is there,Store
34,Distance of Outlet,Store
408,Distance of Shop,Store
352,Distance of store,Store
...,...,...
422,what are the Shop timings,Store
20,what are the store timings,Store
49,when to visit Outlet,Store
76,when to visit Shop,Store


In [ ]:
#lr_clf.predict(feature_gen("I want to Shopping"))[0]

In [ ]:
lr_clf.predict(feature_gen("how to enroll for membership"))[0]

'Loyalty'

In [ ]:
lr_clf.predict(feature_gen("navigate me to near by store"))[0]

'Store'

In [ ]:
lr_clf.predict(feature_gen("peter england red t-shirt"))[0]

'Product'

In [ ]:
lr_clf.predict(feature_gen("store with parking facility"))[0]

'Store'

### Persisting

In [ ]:
import pickle
#import numpy
import tensorflow as tf
from transformers import DistilBertTokenizer, TFDistilBertModel
distil_bert = 'distilbert-base-uncased'
tokenizer = DistilBertTokenizer.from_pretrained(distil_bert)
model = TFDistilBertModel.from_pretrained(distil_bert)
model.save_pretrained('distilbert-base-uncased')
tokenizer.save_pretrained('distilbert-base-uncased')

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_transform', 'activation_13', 'vocab_projector', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


('distilbert-base-uncased/tokenizer_config.json',
 'distilbert-base-uncased/special_tokens_map.json',
 'distilbert-base-uncased/vocab.txt',
 'distilbert-base-uncased/added_tokens.json')

#### Extras

In [ ]:
lr_clf.predict(feature_gen("need to check"))

array(['Store'], dtype=object)

In [ ]:
lr_clf.predict(feature_gen("completely lost"))

array(['Store'], dtype=object)

In [ ]:
lr_clf.predict(feature_gen("what's the need"))

array(['Store'], dtype=object)

In [ ]:
lr_clf.predict(feature_gen("why should I"))

array(['Product'], dtype=object)

In [ ]:
import string

In [ ]:
lr_clf.predict(feature_gen("bill"))

array(['Loyalty'], dtype=object)

#### Persisitng

In [ ]:
import pickle

In [ ]:
# filename = '/dbfs/FileStore/tables/intent_classification/finalized_model_english_after_Datathon.sav'
# pickle.dump(lr_clf, open(filename, 'wb'))

## Loyalty Sub-Intent

In [ ]:
#df = pd.read_csv("/dbfs/FileStore/tables/intent_classification/finalized_questions1.csv")
df = pd.read_csv("finalized_questions1_after_datathon.csv")

In [ ]:
#df.loc[len(df.index)] = ["My loyalty", "Others"]

In [ ]:
df.shape

(2285, 2)

In [ ]:
df.head()

,Text,Sub-intent
0,offer,Coupons
1,Enrol,Others
2,Do you have any offer which are running on pro...,Others
3,What kind of products are available under the ...,Others
4,How can I take this offer?,Others


In [ ]:
df.sort_values('Text',inplace=True, ascending=True)
duplicated_text_series = df.duplicated('Text', keep = False)
df = df[~duplicated_text_series]
print("Total number of text after removing duplicates:", df.shape[0])

Total number of text after removing duplicates: 2079


In [ ]:
df["Text"] = df["Text"].apply(punctuation_removal)

In [ ]:
df["Sub-intent"].value_counts()

Points     755
Coupons    734
Others     590
Name: Sub-intent, dtype: int64

In [ ]:
df["Text"] = df["Text"].astype(str)
df["Sub-intent"] = df["Sub-intent"].astype(str)

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained(distil_bert)
model = TFDistilBertModel.from_pretrained(distil_bert)

All model checkpoint layers were used when initializing TFDistilBertModel.

All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [ ]:
tokenized = df["Text"].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [ ]:
max_len = 50
padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [ ]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(2079, 50)

In [ ]:
input_ids =tf.convert_to_tensor(padded)
attention_mask = tf.convert_to_tensor(attention_mask)

In [ ]:
last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()

In [ ]:
labels = df["Sub-intent"]

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, random_state=42)

In [ ]:
train_features.shape

(1559, 768)

In [ ]:
train_labels.shape

(1559,)

In [ ]:
#from sklearn.naive_bayes import GaussianNB
#lr_clf = GaussianNB()
#lr_clf.fit(train_features, train_labels)

--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-3952795988958885> in <module> 
 1 from sklearn . naive_bayes import GaussianNB
 2 lr_clf = GaussianNB ( ) 
 ----> 3 lr_clf . fit ( train_features , train_labels ) 

 /databricks/python/lib/python3.7/site-packages/sklearn/naive_bayes.py in fit (self, X, y, sample_weight) 
 205 self : object
 206 """
 --> 207 X , y = self . _validate_data ( X , y ) 
 208 y = column_or_1d ( y , warn = True ) 
 209 return self._partial_fit(X, y, np.unique(y), _refit=True,

 AttributeError : 'GaussianNB' object has no attribute '_validate_data'

In [ ]:
lr_clf=LinearSVC(class_weight='balanced',random_state=44)
lr_clf.fit(train_features, train_labels)

/home/deepakchawala/PycharmProjects/HBChawlaProjects/geekeeprojectModules/texttocode/venv/lib/python3.6/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(class_weight='balanced', random_state=44)

In [ ]:
# lr_clf = LogisticRegression(multi_class = "multinomial",random_state=42)
# lr_clf.fit(train_features, train_labels)

In [ ]:
import pickle
filename = '/dbfs/FileStore/tables/intent_classification/finalized_loyalty_subintent_after_datathon.sav'
lr_clf = pickle.load(open(filename, 'rb'))

In [ ]:
lr_clf.score(test_features, test_labels)

0.9230769230769231

In [ ]:
def feature_gen(text):
    text = [i for i in text if i not in string.punctuation]
    text = "".join(text)
    dummy = tokenizer.encode(text,add_special_tokens=True)
    dummy_padded = np.array([dummy + [0]*(max_len-len(dummy))])
    dummy_attention_mask = np.where(dummy_padded != 0, 1, 0)
    dummy_input_ids =tf.convert_to_tensor(dummy_padded)
    dummy_attention_mask = tf.convert_to_tensor(dummy_attention_mask)
    dummy_last_hidden_states = model(dummy_input_ids, attention_mask=dummy_attention_mask)
    dummy_features = dummy_last_hidden_states[0][:,0,:].numpy()
    return dummy_features

In [ ]:
lr_clf.predict(feature_gen("offers"))[0]

'Coupons'

In [ ]:
lr_clf.predict(feature_gen("want to know about redemption of gift vouchers"))[0]

'Coupons'

In [ ]:
lr_clf.predict(feature_gen("Can I generate gift voucher online?"))[0]

'Coupons'

In [ ]:
lr_clf.predict(feature_gen("gift vouchers"))[0]

'Coupons'

In [ ]:
lr_clf.predict(feature_gen("vouchers"))[0]

'Coupons'

In [ ]:
lr_clf.predict(feature_gen("point"))[0]

'Points'

In [ ]:
lr_clf.predict(feature_gen("what different loyalty programme you have"))[0]

'Others'

In [ ]:
lr_clf.predict(feature_gen("on which date did I enroll"))[0]

'Others'

In [ ]:
lr_clf.predict(feature_gen("can I redeem points on above 4000 transactions"))[0]

'Points'

In [ ]:
lr_clf.predict(feature_gen("in which month did I enroll"))[0]

'Others'

In [ ]:
lr_clf.predict(feature_gen("how many rewards will I get"))[0]

'Others'

In [ ]:
lr_clf.predict(feature_gen("rewards"))[0]

'Coupons'

In [ ]:
lr_clf.predict(feature_gen("reward"))[0]

'Coupons'

In [ ]:
lr_clf.predict(feature_gen("super coins"))[0]

'Points'

In [ ]:
lr_clf.predict(feature_gen("how to enroll for loyalty"))[0]

Out[108]: 'Others'

In [ ]:
lr_clf.predict(feature_gen("voucher"))[0]

'Coupons'

### Fails

In [ ]:
lr_clf.predict(feature_gen("any offers for me"))[0]

'Others'

In [ ]:
lr_clf.predict(feature_gen("any discount on amount"))[0]

'Points'

In [ ]:
lr_clf.predict(feature_gen("How can I apply offer"))[0]

'Others'

In [ ]:
lr_clf.predict(feature_gen("any offer on amount"))[0]

'Others'